### **Añadiendo Realismo a datos**
- **Region de venta**

Para iniciar con el realismo de datos vamos a modificar a los paises para que tenga un peso dependiendo del codigo del pais para eso usaremos el script **"Actualizar_Pais.py"**
*(tambien se coloca debajo de este escrito para seguir todo en una lectura)*


In [ ]:
import pandas as pd

# Cargar datos
df_limpio = pd.read_csv("Ventas_IBO_Gunpla_V2.0.csv")
paises = pd.read_csv("Paises con continene.csv").dropna(subset=["id_pais", "continente"])
lista = paises[['id_pais', 'continente']].copy()

# Distribución de pesos por continente
pesos_continente = {
    "Asia": 0.65,        # Fuerte pero no dominante absoluto
    "América": 0.25,     # Incluye EE.UU., LatAm
    "Oceanía": 0.25,     # MISMO nivel que América
    "Europa": 0.08,
    "África": 0.01,
    "Caribe": 0.01
}
lista['peso'] = lista['continente'].map(pesos_continente)

# Reforzar Japón, pero menos extremo
lista.loc[lista['id_pais'] == "JP", 'peso'] *= 30

# Asia Oriental: refuerzo moderado
asia_oriental = ["CN", "KR", "TW", "HK", "PH"]
lista.loc[lista['id_pais'].isin(asia_oriental), 'peso'] *= 3

# EE.UU. con refuerzo fuerte
lista.loc[lista['id_pais'] == "US", 'peso'] *= 8

# México y Brasil con refuerzo moderado
latam = ["MX", "BR"]
lista.loc[lista['id_pais'].isin(latam), 'peso'] *= 2

# África y Caribe casi nulos
lista.loc[lista['continente'] == "África", 'peso'] *= 0.1
lista.loc[lista['continente'] == "Caribe", 'peso'] *= 0.1

# Normalizar pesos
lista['peso'] = lista['peso'] / lista['peso'].sum()

# Asignar países aleatoriamente con los pesos
n = len(df_limpio)
df_limpio['id_pais'] = lista.sample(n=n, weights=lista['peso'], replace=True)['id_pais'].tolist()

# Guardar
df_limpio.to_csv("Ventas_IBO_Gunpla_V3.0.csv", index=False)

print("🌏 Oceanía igual que América, EE.UU. con peso fuerte, Japón dominante, Asia reforzada, África y Caribe casi nulos.")


- **Actualziar productos**
Para esto primero modificaremos el archivo original de nuestros prodcutos para poder regirnos mediante un peso para cada articulo, para eso usaremos el script de **"Actualizar_Prodcuto poner pesos.py"**
*(colocado debajo)*

In [ ]:
import pandas as pd

# Cargar datos
df_prod = pd.read_csv("IBO_Gunplas.csv")

# Asignar peso base = 1
df_prod['peso'] = 1.0

# Peso máximo: Barbatos Lupus Rex
mask_lupusrex = df_prod['nombreProducto'].str.contains("Lupus Rex", case=False)
df_prod.loc[mask_lupusrex, 'peso'] = 50

# Segundo más alto: Barbatos Lupus (pero no Lupus Rex)
mask_lupus = df_prod['nombreProducto'].str.contains("Barbatos Lupus", case=False) & ~mask_lupusrex
df_prod.loc[mask_lupus, 'peso'] = 40

# Luego: Otros Barbatos (pero no Lupus ni Lupus Rex)
mask_barb_otros = df_prod['nombreProducto'].str.contains("Barbatos", case=False) & ~mask_lupus & ~mask_lupusrex
df_prod.loc[mask_barb_otros, 'peso'] = 30

# Normalizar pesos para que sumen 1
df_prod['peso'] /= df_prod['peso'].sum()

# Guardar resultado
df_prod.to_csv("productos_pesados.csv", index=False)
print("Pesos ajustados. Barbatos y sus variantes tienen mayor prioridad.")


- **Agregando los productos nuevos al script**
Una vez hecho el nuevo archivo de productos con pesos (dependiendo de la popularidad del producto) podemos actualizar nuestros datos para que sean mas realistas, para eso usaremos el archivo de **"Actualizar_producto_a_datos.py"**
*(Una vez mas, colocado debajo)* 

In [ ]:
import pandas as pd
import numpy as np

# Cargar ventas y productos con pesos
df_ventas = pd.read_csv("Ventas_IBO_Gunpla_V2.0.csv")
df_productos = pd.read_csv("productos_pesados.csv")  # tu archivo con pesos


# Extraer IDs y pesos
productos = df_productos["id_nuevo"].tolist()
pesos = df_productos["peso"].tolist()

# Generar nueva columna de productos según los pesos
df_ventas["id_producto"] = np.random.choice(
    productos,
    size=len(df_ventas),
    p=pesos
)

# Guardar resultado
df_ventas.to_csv("Ventas_IBO_Gunpla_V3.0.csv", index=False)

print("✅ Productos en ventas actualizados según popularidad.")


## Preparados para el Dashboard

Con este paso, ya contamos con nuestros datos enriquecidos con un **toque de realismo**, listos para dar el siguiente salto: la creación de un **Dashboard** que presente la información de forma **visual** e **interactiva**.

Para ello utilizaremos la herramienta **Power BI**, aprovechando su capacidad para transformar datos en visualizaciones impactantes y fáciles de interpretar.

➡️ Ir a **"3. Dashboard"** para continuar.

